# Epileptic Seizure Classification with an Autoencoder - Threshold Method
This notebook contains the classification of time series EEG data for the detection of epileptic seizures based on the preprocessed CHB-MIT Scalp EEG Database. <br>
The codes is structured as followed:
1. [Imports](#1-imports)
2. [Load Preprocessed Dataset](#2-load-preprocessed-dataset)
3. [Split Dataset](#3-split-dataset)
4. [Normalize Dataset](#4-normalize-dataset)
5. [Autoencoder](#5-autoencoder) <br>
5.1 [Seperate Normal & Anomaly Data](#51-seperate-normal--anomaly-data) <br>
5.2 [Define Autoencoder-Model](#52-define-autoencoder-model) <br>
5.3 [Compile Autoencoder-Model](#53-compile-autoencoder-model) <br>
5.4 [Train Autoencoder](#54-train-autoencoder-model) <br>
5.5 [Visualize Reconstruction](#55-visualize-reconstruction)
6. [Visualize Reconstruction Error](#6-visualize-reconstruction-error)
7. [Binary Classification](#7-binary-classification)
8. [Conclusions](#8-conclusion)

## 1. Imports
Import requiered libraries. <br>
External packages can be installed via the `pip install -r requirements.txt` command or the notebook-cell below.

In [ ]:
! pip install -r ../requirements.txt

In [1]:
# Import built-in libraries
import warnings
warnings.filterwarnings("ignore", message=".*The 'nopython' keyword.*") # Suppress SHAP warnings

# Import datascience libraries
import numpy as np

# Import preprocessing-libraries, classification metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from imblearn.metrics import geometric_mean_score

# Import visualization libraries
import plotly.express as px
import plotly.graph_objects as go
from prettytable import PrettyTable

# Neural Network
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv1D, MaxPooling1D, UpSampling1D
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Import explainability library
import shap

## 2. Load Preprocessed Dataset
In order to load the preprocessed dataset, that was created with the notebook [00_Preprocessing.ipynb](../01_Data%20Preprocessing/00_Preprocessing.ipynb), is loaded and the numpy Arrays for the features and labels are extracted. <br>
To enshure a functional distribution of the classes in the dataset, the classes with the respective amounts are plotted.

In [2]:
dataset = np.load('../00_Data/Processed-Data/classification_dataset_max.npz')
X = dataset["features"]
y = dataset["labels"]

In [3]:
print("Shapes: ", X.shape, y.shape)
print(np.unique(y, return_counts=True))

Shapes:  (16999, 1000, 24) (16999, 1)
(array([0, 1], dtype=int8), array([9375, 7624]))


## 3. Split Dataset
In order to validate and test the trained classifier, the dataset must be split into a `train`, `test`, and `validation` subset. <br>
To preserve an equal distribution within each split, the `stratify`-option is enabled.

In [4]:
X_train, X_rest, y_train, y_rest = train_test_split(X, y, test_size=0.4, shuffle=True, stratify=np.ravel(y), random_state=34)
X_test, X_val, y_test, y_val = train_test_split(X_rest, y_rest, test_size=0.5, shuffle=True, stratify=np.ravel(y_rest), random_state=34)

## 4. Normalize Dataset
When working with neural networks, it is imperative to normalize the data bevore training and testing. This enshures a faster training, avoids numerical instablities and provides a better generalization of the neural network. However with EEG-data, there are additional requirements due to the different characteristics and value-ranges of the individual channels. Therefore, the normalization is done channel by channel based on the training-subset and applied on the test- and validation-split.

In [5]:
def normalize_features(X_train:np.ndarray, X_test:np.ndarray, X_val:np.ndarray, use_standard_scaler:bool=False) -> tuple:
    if(use_standard_scaler):
        scaler = StandardScaler() # Create Z-Score normalizer
    else:
        scaler = MinMaxScaler() # Create Min-Max normalizer
    X_train_norm = np.zeros(shape=(X_train.shape), dtype='float32') # Create empty array for normalized train-data
    X_test_norm = np.zeros(shape=(X_test.shape), dtype='float32') # Create empty array for normalized test-data
    X_val_norm = np.zeros(shape=(X_val.shape), dtype='float32') # Create empty array for normalized val-data
    for feature_col in range(X_train.shape[2]): # Iterate over features in dataset
        X_train_norm[:,:,feature_col] = scaler.fit_transform(X_train[:,:,feature_col]) # Fit and apply normalizer on current feature in train subset
        X_test_norm[:,:,feature_col] = scaler.transform(X_test[:,:,feature_col]) # Apply normalizer on current feature in test subset
        X_val_norm[:,:,feature_col] = scaler.transform(X_val[:,:,feature_col]) # Apply normalizer on current feature in val subset
    return X_train_norm, X_test_norm, X_val_norm

In [6]:
X_train_normalized, X_test_normalized, X_val_normalized = normalize_features(X_train, X_test, X_val, True)

## 5. Autoencoder
The following section contains the data-preperation, build and training of the autoencoder. <br>

<b>What is an autoencoder?</b><br>
An autoencoder is a neural network architecture, that is used for unsupervised machine learning tasks. It consists out of two main components: The encoder & decoder. The encoder takes the input data and transforms it into a lower dimensional representation of the data, the so-called "latent-space". The decoder-part takes this data and tries to reconstruct the original input data. The main target during the training-phase is to minimize the reconstruction error. <br>

<b>How can autoencoders be used for the detection of epileptic seizures in EEG-data?</b><br>
There are two options how autoencoders can be used for the detection of epileptic seizures in EEG-data: Reconstruction-Error & Latent-Space. <br>
By training the autoencoder only on data that does not contain any epileptic seizures, the reconstruction error for "normal" data is minimized. That means that if a sample with an active seizure is predicted, the reconstruction error will be increased. By defining an error-threshold, a binary classification can be performed to seperate normal samples from samples with an epileptic seizure.

The second option is to use the latent space for the classification. The autoencoder is trained on the complete dataset with the same task of minimizing the reconstruction error. By seperating the decoding-component from the autoencoder, the latent space is exposed. Because of the differences in the data when an epileptic seizure is present, the representation of these samples must be different in the reduced space. Based on this assumption, a classification by using a clustering-approach can be done.

The following code contains the first approach.

### 5.1 Seperate Normal & Anomaly Data
In order to detect anomalies in the EEG-data that indicate an epileptic seizure, the autoencoder is only trained on samples where no seizure is present. Therefore, the samples must be seperated into "normal" and "anomalie" data. 

In [7]:
X_train_normal = X_train_normalized[np.where(y_train == 0)[0]]
X_train_anomalies = X_train_normalized[np.where(y_train == 1)[0]]

X_val_normal = X_val_normalized[np.where(y_val == 0)[0]]
X_val_anomalies = X_val_normalized[np.where(y_val == 1)[0]]

### 5.2 Define Autoencoder-Model
As described, an autoencoder consists out of a encoder and decoder component. The encoder reduces the dimensionality of the data and the decoder tries to reconstruct the input data. The resulting neural network model can be defined as follows:

In [8]:
def build_and_compile_model(train_shape:tuple, initial_lr:float=0.0001):
    inputs = Input(shape=(train_shape[1], train_shape[2]))
    # Encoder
    E1 = Conv1D(filters=24, kernel_size=3, activation='relu', padding='same')(inputs)
    E2 = MaxPooling1D(pool_size=2, padding='same')(E1)
    E3 = Conv1D(filters=12, kernel_size=3, activation='relu', padding='same')(E2)
    E4 = MaxPooling1D(pool_size=2, padding='same')(E3)
    latent_vector = Conv1D(filters=6, kernel_size=3, activation='relu', padding='same')(E4)
    # Decoder
    D1 = Conv1D(filters=6, kernel_size=3, activation='relu', padding='same')(latent_vector)
    D2 = UpSampling1D(size=2)(D1)
    D3 = Conv1D(filters=12, kernel_size=3, activation='relu', padding='same')(D2)
    D4 = UpSampling1D(size=2)(D3)
    D5 = Conv1D(filters=24, kernel_size=3, activation='relu', padding='same')(D4)
    ae_outputs = Dense(train_shape[2])(D5)

    autoencoder = Model(inputs=inputs, outputs=ae_outputs)
    opt = tf.keras.optimizers.legacy.Adam(learning_rate=initial_lr)

    autoencoder.compile(optimizer=opt, loss='mse')
    return autoencoder

### 5.3 Compile Autoencoder-Model
After defining the neural network, the model must be compiled. In addition the optimizer and loss-function must be set.

In [10]:
autoencoder = build_and_compile_model(X_train_normal.shape, 0.001)

earlystopper_ae = EarlyStopping(monitor='val_loss', patience=10, start_from_epoch=10, restore_best_weights=True, verbose=1)
reduce_lr_ae = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0000001, verbose=1, cooldown=10)

autoencoder.summary()

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1000, 24)]        0         
                                                                 
 conv1d (Conv1D)             (None, 1000, 24)          1752      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 500, 24)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 500, 12)           876       
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 250, 12)          0         
 1D)                                                             
                                              

### 5.4 Fit Autoencoder-Model
Fitting the neural network stands for the training process of the weights and bias of each layer to enable good predictions. The normalized training data is used for the training step and a validation is performed after every epoch via the normalized validation split to detect and prevent overfitting. Different to a "normal" training process using the features and labels, the feature-data is used for both input and output. After training, the progression of the training and validation loss is visualized for the easy visualization of the fit-process and further detection of overfitting or other issues during the training. In addition, an example of the original and reconstructed data is plottet to see the capabilities of the autoencoder. Finally, the model is saved as an .h5 file to be able to load the model an/or weights during later testing without new training.

In [11]:
history = autoencoder.fit(
    X_train_normal, 
    X_train_normal,
    epochs=500,
    batch_size=50,
    validation_data=(X_val_normal, X_val_normal),
    shuffle=True,
    verbose=1,
    callbacks=[earlystopper_ae, reduce_lr_ae]
)

Epoch 1/250


2023-07-02 01:47:24.935278: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


113/113 [==============================] - 23s 149ms/step - loss: 0.3749 - val_loss: 0.3188 - lr: 0.0010
Epoch 2/250
113/113 [==============================] - 14s 118ms/step - loss: 0.3032 - val_loss: 0.2755 - lr: 0.0010
Epoch 3/250
113/113 [==============================] - 12s 104ms/step - loss: 0.2604 - val_loss: 0.2486 - lr: 0.0010
Epoch 4/250
113/113 [==============================] - 13s 111ms/step - loss: 0.2454 - val_loss: 0.2388 - lr: 0.0010
Epoch 5/250
113/113 [==============================] - 13s 112ms/step - loss: 0.2372 - val_loss: 0.2288 - lr: 0.0010
Epoch 6/250
113/113 [==============================] - 12s 109ms/step - loss: 0.2265 - val_loss: 0.2208 - lr: 0.0010
Epoch 7/250
113/113 [==============================] - 12s 108ms/step - loss: 0.2207 - val_loss: 0.2167 - lr: 0.0010
Epoch 8/250
113/113 [==============================] - 12s 108ms/step - loss: 0.2179 - val_loss: 0.2138 - lr: 0.0010
Epoch 9/250
113/113 [==============================] - 11s 101ms/step - loss

In [12]:
fig = go.Figure(
    data = [
        go.Scatter(y=history.history['loss'], name="train"),
        go.Scatter(y=history.history['val_loss'], name="val"),
    ],
    layout = {"yaxis": {"title": "Loss [MSE]"}, "xaxis": {"title": "Epoch"}, "title": "Reconstruction Loss over Epochs"}
)

fig.show()

In [13]:
autoencoder.save('../99_Assets/01_Saved Models/02_autoencoder_threshold.h5')
# autoencoder = tf.keras.models.load_model('../99_Assets/01_Saved Models/02_autoencoder_threshold.h5')

### 5.5 Visualize Reconstruction

In [14]:
a = np.transpose(X_train_normal, (0,2,1))
data = []
for i in a[0]:
    data.append(
        go.Scatter(y=i)
    )

fig = go.Figure(
    data = data,
    layout = {"yaxis": {"title": "Loss [MSE]"}, "xaxis": {"title": "Epoch"}, "title": "Model Loss over Epochs"}
)

fig.show()

In [15]:
pred = autoencoder.predict(X_train_normal[:1])
a = np.transpose(pred, (0,2,1))
data = []
for i in a[0]:
    data.append(
        go.Scatter(y=i)
    )

fig = go.Figure(
    data = data,
    layout = {"yaxis": {"title": "Loss [MSE]"}, "xaxis": {"title": "Epoch"}, "title": "Model Loss over Epochs"}
)

fig.show()

1/1 [==============================] - 0s 482ms/step


## 6. Visualize Reconstruction Error

In [16]:
X_val_pred_normal = autoencoder.predict(X_val_normal)
mse = np.mean(np.square(X_val_normal - X_val_pred_normal), axis=(1, 2))
mae = np.abs(np.mean(X_val_normal - X_val_pred_normal, axis=(1,2)))
fig = px.histogram(x=mae, nbins=400)
fig.show()

59/59 [==============================] - 2s 31ms/step


In [17]:
X_val_pred_anomalies = autoencoder.predict(X_val_anomalies)
mse = np.mean(np.square(X_val_anomalies - X_val_pred_anomalies), axis=(1, 2))
mae = np.abs(np.mean(X_val_anomalies - X_val_pred_anomalies, axis=(1,2)))
fig = px.histogram(x=mae, nbins=400)
fig.show()

48/48 [==============================] - 1s 31ms/step


## 7. Binary Classification

In [18]:
X_test_pred = autoencoder.predict(X_test_normalized)
mse = np.mean(np.square(X_test_pred - X_test_normalized), axis=(1, 2))
mae = np.abs(np.mean(X_test_pred - X_test_normalized, axis=(1,2)))

107/107 [==============================] - 3s 30ms/step


In [98]:
y_test_predictions = np.where(mae >= 0.00512, 1, 0)

In [99]:
f1score = f1_score(y_test, y_test_predictions)
gm = geometric_mean_score(y_test, y_test_predictions, average="binary")
auc = roc_auc_score(y_test, y_test_predictions, average="weighted")
precision = precision_score(y_test, y_test_predictions)
recall = recall_score(y_test, y_test_predictions)

In [100]:
data = [["F1-Score", "G-Mean", "AUC", "Precision", "Recall"], [f1score, gm, auc, precision, recall]]
table = PrettyTable(data[0])
table.add_rows(data[1:])
print(table)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|      F1-Score      |       G-Mean       |        AUC         |     Precision      |       Recall       |
+--------------------+--------------------+--------------------+--------------------+--------------------+
| 0.6558843771507227 | 0.6944566771964485 | 0.6983256830601093 | 0.6900796524257784 | 0.6249180327868853 |
+--------------------+--------------------+--------------------+--------------------+--------------------+


## 8. Conclusion